<a href="https://colab.research.google.com/github/rushikeshnakhate/NSESectorAnalysis/blob/main/nsesectorsanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install jugaad_data -q
!pip install dataclass -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.8/115.8 kB 5.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 2.2.5 requires click>=8.0, but you have click 7.1.2 which is incompatible.
dask 2023.8.1 requires click>=8.0, but you have click 7.1.2 which is incompatible.
distributed 2023.8.1 requires click>=8.0, but you have click 7.1.2 which is incompatible.
fiona 1.9.6 requires click~=8.0, but you have click 7.1.2 which is incompatible.
pip-tools 6.13.0 requires click>=8, but you have click 7.1.2 which is incompatible.
yfinance 0.2.38 requires beautifulsoup4>=4.11.1, but you have beautifulsoup4 4.9.3 which is incompatible.
ERROR: Could not find a version that satisfies the requirement dataclass (from versions: none)
ERROR: No matching distribution found for d

In [ ]:
from pathlib import Path
from datetime import date
from jugaad_data.nse import NSELive
from jugaad_data.nse import bhavcopy_save
import pandas as pd
import os
import pickle
import shutil
from datetime import date, timedelta
from dataclasses import dataclass
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns


output_dir = Path("/content/drive/MyDrive")
sectorFile = output_dir.joinpath("Sector.csv")
holidayFile = output_dir.joinpath("Holiday.csv")
correlationFile= output_dir.joinpath("Correlation.csv")


In [ ]:
def validate_required_args(kwargs, *required_args):
    for arg in required_args:
        if arg not in kwargs:
            raise ValueError(f"Error: '{arg}' argument not provided, actual received: {kwargs}")

def load_or_download_and_cache_data(download_function:callable,**kwargs):
  validate_required_args(kwargs, 'cache_file')
  cache_file : str = kwargs.get("cache_file")

  if os.path.exists(cache_file):
    with open(cache_file,'rb') as file:
      return pickle.load(file)

  data_df = download_function(**kwargs)
  # if not data_df.empty:
  with open(cache_file,'wb') as file:
    pickle.dump(data_df,file)
  return data_df


In [ ]:
def download_bhavcopy(**kwargs):
  """
  # bhavcopy_save: Generates a CSV file with a random name.
  # The expected name for the downloaded Bhavcopy file is "bhavcopy_{start_date}.csv". # The cache file is named as "bhavcopy_
{start_date}.pkl". A renaming process is required.
  """
  validate_required_args(kwargs, 'cache_file','start_date')
  cache_file: str = kwargs.get("cache_file")
  start_date : date= kwargs.get("start_date")

  generated_csv_filename = bhavcopy_save(start_date, output_dir)
  cache_file_to_csv_name = cache_file.with_suffix(".csv")
  os.rename(generated_csv_filename,cache_file_to_csv_name)
  return pd.read_csv(cache_file_to_csv_name)


In [ ]:
def fetch_or_genenate_stocks(start_date: date):
  output_file  = Path(output_dir)/("NseData_" + str(start_date) +".pkl")
  return load_or_download_and_cache_data(download_bhavcopy,start_date= start_date,cache_file=output_file)

def get_sectors():
    sector_df = pd.read_csv(sectorFile)
    return sector_df

In [ ]:
def fetch_or_generate_sector_to_stocks(stocks : pd.DataFrame):
  output_file  = Path(output_dir)/("Sectors.pkl")
  sector_df = get_sectors()
  # display(stocks_to_sector_map_df)
  try:
   return pd.merge(stocks, sector_df, on='SYMBOL', how='inner',suffixes=('', ''))
  except pd.errors.MergeError as e:
    print(f"Merge failed: {e}")

In [ ]:
def calculate_performace(stocks_df : pd.DataFrame, start_date: date):
    # Calculate daily returns
  print("calculate_performace for date={}".format(start_date))


  stocks_df['DAILY_RETURN'] =(stocks_df['CLOSE'] - stocks_df['OPEN'])/ stocks_df['OPEN']
  # Group by 'SECTOR' and calculate average daily return and volatility
  sector_performance_df = stocks_df.groupby('SECTOR').agg({
      # stocks_list=('STOCK', lambda x: list(x))
      'DAILY_RETURN': ['std','mean','count']
  }).reset_index()

  sector_performance_df["TRADEDATE"] = start_date

  # Flatten the column names, but only for columns where the second part is not an empty string
  sector_performance_df.columns = [
      f"{col[0]}_{col[1]}" if col[1] else col[0] for col in sector_performance_df.columns
  ]
  return sector_performance_df


def prepare_stocks_data_and_calculate_performance(**kwargs):
  try:
    validate_required_args(kwargs, 'start_date')
    start_date : date = kwargs.get('start_date')
    stocks_df = fetch_or_genenate_stocks(start_date)
    stocks_df_with_sector = fetch_or_generate_sector_to_stocks(stocks_df)
    return calculate_performace(stocks_df_with_sector,start_date)
  except ValueError as e:
    print(e)

def fetch_or_generate_performance_for_stocks(start_date:date):
  sector_performance_output_file = Path(output_dir)/("SectorPeformance_" + str(start_date) +".pkl")
  return load_or_download_and_cache_data(prepare_stocks_data_and_calculate_performance,start_date=start_date,cache_file=sector_performance_output_file)


In [ ]:
def is_weekend(date : date):
    day = date.weekday()
    if day == 5 or day == 6:
        return True

def is_public_holiday(date : date):
    public_holidays_df = pd.read_csv(holidayFile)
    return date.strftime('%d-%m-%Y') in public_holidays_df['Date'].values


In [ ]:
def get_uniq_sectors():
  sectors_csv = pd.read_csv(sectorFile)
  sectors = sectors_csv['SECTOR'].unique()
  sectors_stripped = [sector.strip() for sector in sectors]
  unique_sectors = list(set(sectors_stripped))
  return unique_sectors


In [ ]:
@dataclass
class CorrelationData:
    #Date: date
    FirstSector: str
    SecondSector: str
    CorrelationByMean: float
    CorrelationByDeviation: float

def populate_sectors_correlation_df(generate_performance_for_stocks_group_by_sector):
    correlation_data_list = []
    sector_list_mean_matrix = {}
    sector_list_dev_matrix = {}
    dataframe_correlation = pd.DataFrame()

    sectors = get_uniq_sectors()
    for sector in sectors:
        sector_list_mean_matrix[sector] = []
        sector_list_dev_matrix[sector] = []

    if not generate_performance_for_stocks_group_by_sector.empty and not generate_performance_for_stocks_group_by_sector.empty:
      for sector in sectors:
        for index, row in generate_performance_for_stocks_group_by_sector.iterrows():
          if row['SECTOR']  ==  sector:
             sector_list_mean_matrix[sector].append(row['DAILY_RETURN_mean'])
             sector_list_dev_matrix[sector].append(row['DAILY_RETURN_std'])
    dataframe_correlation = pd.DataFrame(sector_list_mean_matrix)


    for i in range(len(sectors)):
         for j in range(i + 1, len(sectors)):
            first_sector = sectors[i]
            second_sector = sectors[j]
            #print(first_sector,second_sector )
            correlation_mean = pd.Series(sector_list_mean_matrix[first_sector]).corr(pd.Series(sector_list_mean_matrix[second_sector]))
            correlation_std = pd.Series(sector_list_dev_matrix[first_sector]).corr(pd.Series(sector_list_dev_matrix[second_sector]))
            correlation_data = CorrelationData(FirstSector=first_sector, SecondSector=second_sector, CorrelationByMean=correlation_mean, CorrelationByDeviation=correlation_std)
            correlation_data_list.append(correlation_data)

    correlation_df = pd.DataFrame([vars(x) for x in correlation_data_list])
    return  correlation_df

# Better version of above populate_sectors_correlation_df
def populate_sectors_correlation_df_1(generate_performance_for_stocks_group_by_sector , correlation_method= "pearson"):

  if correlation_method not in ["pearson", "spearman", "kendall"]:
    raise RuntimeError("Invalid correlation method. Supported methods are 'Pearson', 'Spearman', and 'Kendall'.")


  correlation_data_list = []
  sector_list_mean_matrix = {}
  sector_list_dev_matrix = {}
  sectors = get_uniq_sectors()

  for sector in sectors:
      sector_list_mean_matrix[sector] = []
      sector_list_dev_matrix[sector] = []

  if not generate_performance_for_stocks_group_by_sector.empty and not generate_performance_for_stocks_group_by_sector.empty:
      for sector in sectors:
          sector_data = generate_performance_for_stocks_group_by_sector[generate_performance_for_stocks_group_by_sector['SECTOR'] == sector]
          sector_list_mean_matrix[sector] = sector_data['DAILY_RETURN_mean'].tolist()
          sector_list_dev_matrix[sector] = sector_data['DAILY_RETURN_std'].tolist()

  for i in range(len(sectors)):
      for j in range(i + 1, len(sectors)):
          first_sector = sectors[i]
          second_sector = sectors[j]

          if correlation_method == "pearson":
            correlation_mean = pd.Series(sector_list_mean_matrix[first_sector]).corr(pd.Series(sector_list_mean_matrix[second_sector]))
            correlation_std = pd.Series(sector_list_dev_matrix[first_sector]).corr(pd.Series(sector_list_dev_matrix[second_sector]))
          elif correlation_method == "spearman":
            correlation_mean = pd.Series(sector_list_mean_matrix[first_sector]).corr(pd.Series(sector_list_mean_matrix[second_sector]),method='spearman')
            correlation_std = pd.Series(sector_list_dev_matrix[first_sector]).corr(pd.Series(sector_list_dev_matrix[second_sector]),method='spearman')
          elif correlation_method == "kendall":
            correlation_mean = pd.Series(sector_list_mean_matrix[first_sector]).corr(pd.Series(sector_list_mean_matrix[second_sector]),method='kendall')
            correlation_std = pd.Series(sector_list_dev_matrix[first_sector]).corr(pd.Series(sector_list_dev_matrix[second_sector]),method='kendall')
          # correlation_mean = pd.Series(sector_list_mean_matrix[first_sector]).corr(pd.Series(sector_list_mean_matrix[second_sector]))
          # correlation_std = pd.Series(sector_list_dev_matrix[first_sector]).corr(pd.Series(sector_list_dev_matrix[second_sector]))
          correlation_data_list.append({'FirstSector': first_sector, 'SecondSector': second_sector, 'CorrelationByMean': correlation_mean, 'CorrelationByDeviation': correlation_std})

  correlation_df = pd.DataFrame(correlation_data_list)
  return correlation_df


In [ ]:
from datetime import date, timedelta
import calendar

def generate_dates(year, start_month, end_month):
    start_date = date(year, start_month, 1)
    # Find the last day of the end month
    _, last_day = calendar.monthrange(year, end_month)
    end_date = date(year, end_month, last_day)

    current_date = start_date
    extended_dates = []
    while current_date <= end_date:
        extended_dates.append(current_date)
        current_date += timedelta(days=1)
    return extended_dates



def plot (correlation_df):
  plt.figure(figsize=(12, 8))
  sns.heatmap(correlation_df.reindex(index=sectors, columns=sectors), annot=True, cmap='coolwarm', fmt=".2f")
  plt.title('Correlation Heatmap between Sectors')
  plt.xlabel('SecondSector')
  plt.ylabel('FirstSector')
  plt.xticks(rotation=90)
  plt.yticks(rotation=0)
  plt.tight_layout()
  plt.show()


def  get_corelation_of_one_sector_with_all_by(column,correlation_df):
  correlation_matrix = {}
  for index, row in correlation_df.iterrows():
    if row['FirstSector'] not in correlation_matrix:
        correlation_matrix[row['FirstSector']] = {}
    correlation_matrix[row['FirstSector']][row['SecondSector']] = row[column]

    if row['SecondSector'] not in correlation_matrix:
        correlation_matrix[row['SecondSector']] = {}
    correlation_matrix[row['SecondSector']][row['FirstSector']] = row[column]

  for sector in correlation_matrix.keys():
    correlation_matrix[sector][sector] = 1

  correlation_df = pd.DataFrame(correlation_matrix)
  return correlation_df


In [ ]:
import datetime
if __name__ == "__main__":
  startdates = []
  for year in range(2023, 2013, -1):  # Loop from 2023 to 2013 (inclusive)
    startdates.extend(generate_dates(year, 1, 12))

  startdates = list(set(startdates))
  dfs = []
  for startdate in startdates:
    if is_weekend(startdate) or is_public_holiday(startdate):
        continue
    try:
        generate_performance_for_stocks_group_by_sector = fetch_or_generate_performance_for_stocks(startdate)
        stocks = fetch_or_genenate_stocks(startdate)
        stocks_wth_sectors = fetch_or_generate_sector_to_stocks(stocks)
        dfs.append(generate_performance_for_stocks_group_by_sector)
    except Exception as e:
        print(f"Failed to generate performance for {startdate}: {e}")
        continue


  correlationMethods = ["pearson","spearman","kendall"]
  for correlationMethod in correlationMethods:
    combined_df = pd.concat(dfs, ignore_index=True)
    correlation_df = populate_sectors_correlation_df_1(combined_df,correlationMethod)
    correlation_matrix_by_mean = get_corelation_of_one_sector_with_all_by('CorrelationByMean',correlation_df)
    correlation_matrix_deviation = get_corelation_of_one_sector_with_all_by('CorrelationByDeviation',correlation_df)
    display(correlation_matrix_by_mean)
    display(correlation_matrix_deviation)


Failed to generate performance for 2021-11-19: File is not a zip file
Failed to generate performance for 2019-08-12: File is not a zip file
Failed to generate performance for 2017-08-25: File is not a zip file


In [ ]:
# NSE Index Performance:
# Trend Observation: Look at the overall trend of the NSE Index line. An upward trend suggests a positive week for the market, whereas a downward trend indicates a bearish market. Stability or minor fluctuations might suggest a period of consolidation.
# Conclusion: If the index shows a significant upward trend, you could conclude that the market was optimistic during that week. Conversely, a downward trend could indicate market pessimism or reaction to adverse events.
# Sector Mean Returns:
# Performance Comparison: Identify which sectors had the highest mean returns. High positive mean returns suggest sectors that outperformed during the week.
# Conclusion: Sectors with the highest mean returns were the leading performers and potentially received positive news or investor sentiment. Sectors with negative returns might have faced challenges or negative sentiment.
# Sector Volatility:
# Risk Assessment: Volatility indicates risk. Higher volatility suggests greater uncertainty or variability in returns.
# Conclusion: Sectors with high volatility experienced more significant price movements and could be considered riskier investments during that week. Lower volatility suggests stability, potentially making them safer choices for risk-averse investors.
# Sector Weighted Average Returns:
# Impact of Key Stocks: This metric shows the overall sector performance considering the size or importance of stocks (e.g., by trading volume).
# Conclusion: If certain sectors show high weighted average returns, it indicates that larger, possibly more influential stocks performed well, driving the sector's overall performance. Comparing weighted average returns to mean returns can reveal the impact of larger stocks on sector performance.
# General Conclusions:
# Market Overview: The combined view of the NSE Index and sector performance metrics offers a comprehensive market overview, highlighting overall market trends, sector-specific performances, and risk profiles.
# Sector Analysis: By examining all metrics together, you can identify sectors that are not only performing well but are also less volatile, suggesting a favorable risk-return profile. Conversely, sectors with poor or volatile performance might warrant caution or further investigation.
# Investment Insights: Investors might find opportunities in sectors with strong, stable performance. High volatility sectors might offer short-term trading opportunities but come with higher risk.
# Limitations:
# Short Time Frame: Conclusions drawn from a week's data are inherently limited and should be considered in the context of longer-term trends and external factors (e.g., economic news, global events).
# Data Depth: Additional insights could be gained from more detailed data, such as price-to-earnings ratios, dividend yields, or macroeconomic indicators, for a fuller analysis.
# These conclusions are based on hypothetical data and general principles of financial analysis. Actual data might lead to different interpretations, and any investment decisions should be made considering comprehensive analysis and personal investment goals.